In [1]:
import pandas as pd
import pandas_ta as ta
import yfinance as yf
#import matplotlib.pyplot as plt
from datetime import datetime,timedelta

In [2]:
df= pd.read_csv("./data/nifty500_daily_01Jan_26Jul2023.csv")
df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0',], inplace=True)

In [3]:
df_30m = pd.read_csv("./data/nifty_from_2023-06-01_to_2023-07-25-by30m.csv")


In [4]:
# Calculate the Simple Moving Average (SMA) with a window of 20,30,40,50 periods
df['SMA20'] = ta.sma(df['Close'], length=20)

df['SMA30'] = ta.sma(df['Close'], length=30)

df['SMA40'] = ta.sma(df['Close'], length=40)

df['SMA50'] = ta.sma(df['Close'], length=40)

df['SMA200'] = ta.sma(df['Close'], length=200)
# Calculate the Relative Strength Index (RSI) with a window of 14 periods
df['RSI'] = ta.rsi(df['Close'], length=14)



In [5]:
#df.iloc[50:65]

In [6]:
def check_symbol(input_symbol: str) -> str:
    # Check if last part is ".NS"
    if not input_symbol.endswith(".NS"):
        input_symbol += ".NS"

    # Check if the whole symbol is in capital letters
    if not input_symbol.isupper():
        input_symbol = input_symbol.upper()

    # Check if the symbol is present under the 'symbol' column of df
    if input_symbol not in df['symbol'].unique():
        raise ValueError(f"'{input_symbol}' is  not in NIFTY 500.")

    return input_symbol


In [7]:
def ma_crossover(symbol: str, band: int=5, start_date: str = None, end_date: str = None,
                 oversold: int = 50, overbought: int = 75) -> None:

    symbol = check_symbol(symbol)

    # If start_date is not provided, set it to 3 months prior to the current date
    if not start_date:
        start_date = (datetime.now() - timedelta(days=3 * 30)).strftime('%Y-%m-%d')

    # If end_date is not provided, set it to the current date
    if not end_date:
        end_date = datetime.now().strftime('%Y-%m-%d')
    # Select rows for symbol
    selected_rows = df[df['symbol'] == symbol]

    # Calculate 0.25% of the Close price
    crossover = band / 100 * selected_rows['Close']

    # Check if SMA20, SMA30, SMA40, and SMA50 are within certain Close price
    within_band = (
        (selected_rows['SMA20'] >= (selected_rows['Close'] - crossover)) &
        (selected_rows['SMA20'] <= (selected_rows['Close'] + crossover)) &
        (selected_rows['SMA30'] >= (selected_rows['Close'] - crossover)) &
        (selected_rows['SMA30'] <= (selected_rows['Close'] + crossover)) &
        (selected_rows['SMA40'] >= (selected_rows['Close'] - crossover)) &
        (selected_rows['SMA40'] <= (selected_rows['Close'] + crossover)) &
        (selected_rows['SMA50'] >= (selected_rows['Close'] - crossover)) &
        (selected_rows['SMA50'] <= (selected_rows['Close'] + crossover)) &
        (selected_rows['RSI'].between(oversold, overbought)) &
        (selected_rows['date'].between(start_date, end_date))
    )

    # Filter the selected rows based on the condition
    selected_rows = selected_rows[within_band]

    # Convert the 'date' column to datetime type
    selected_rows['date'] = pd.to_datetime(selected_rows['date'])

    # Get the row with the earliest date in the selected rows
    earliest_date_row = selected_rows.iloc[selected_rows['date'].argsort()[:1]]

    # Get the row with the latest date in the selected rows
    latest_date_row = selected_rows.iloc[selected_rows['date'].argsort()[-1:]]

    # Count of selected rows
    num_selected_rows = len(selected_rows)

    # Print the count of selected rows and the resulting DataFrames
    print(f"Cross Over observed for {symbol} : {num_selected_rows} times in this period ")

    # Print the resulting DataFrames

    print(f"Crossover for {symbol } observed on the earliest date:")
    print(earliest_date_row[['symbol', 'Close', 'date', 'RSI']])

    print(f"\nCross over for {symbol} observed on  the latest date:")
    print(latest_date_row[['symbol', 'Close', 'date', 'RSI']])
    #print(latest_date_row.to_string(index=False))
     


In [8]:
ma_crossover('TCS')
#help(ma_crossover)

Cross Over observed for TCS.NS : 19 times in this period 
Crossover for TCS.NS observed on the earliest date:
       symbol        Close       date        RSI
46892  TCS.NS  3298.699951 2023-05-22  60.515458

Cross over for TCS.NS observed on  the latest date:
       symbol    Close       date      RSI
46925  TCS.NS  3329.25 2023-07-07  60.5295


In [9]:
ma_crossover('PAYTM')

Cross Over observed for PAYTM.NS : 8 times in this period 
Crossover for PAYTM.NS observed on the earliest date:
         symbol       Close       date        RSI
39284  PAYTM.NS  706.150024 2023-05-26  55.539334

Cross over for PAYTM.NS observed on  the latest date:
         symbol       Close       date        RSI
39291  PAYTM.NS  709.549988 2023-06-06  55.476211


In [10]:
ma_crossover('InFY')

Cross Over observed for INFY.NS : 22 times in this period 
Crossover for INFY.NS observed on the earliest date:
        symbol        Close       date        RSI
19872  INFY.NS  1292.800049 2023-05-22  52.411601

Cross over for INFY.NS observed on  the latest date:
        symbol        Close       date        RSI
19905  INFY.NS  1330.199951 2023-07-07  58.010087


In [11]:
ma_crossover('COCHINSHIP')


Cross Over observed for COCHINSHIP.NS : 5 times in this period 
Crossover for COCHINSHIP.NS observed on the earliest date:
              symbol   Close       date        RSI
48682  COCHINSHIP.NS  539.75 2023-06-09  56.667278

Cross over for COCHINSHIP.NS observed on  the latest date:
              symbol  Close       date        RSI
48699  COCHINSHIP.NS  562.0 2023-07-05  51.995462


In [12]:
dir(ta)

['ALL_PATTERNS',
 'AllStrategy',
 'AnalysisIndicators',
 'BasePandasObject',
 'CANGLE_AGG',
 'CDL_PATTERN_NAMES',
 'Category',
 'CommonStrategy',
 'DataFrame',
 'DistributionNotFound',
 'EXCHANGE_TZ',
 'Imports',
 'List',
 'Optional',
 'PandasObject',
 'Path',
 'Pool',
 'RATE',
 'Series',
 'Strategy',
 'Timedelta',
 'Timestamp',
 'Tuple',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_dist',
 'aberration',
 'above',
 'above_value',
 'accbands',
 'ad',
 'adosc',
 'adx',
 'alma',
 'alphavantage',
 'amat',
 'ao',
 'aobv',
 'apo',
 'argmax',
 'argmin',
 'aroon',
 'atr',
 'av',
 'bbands',
 'below',
 'below_value',
 'bias',
 'bop',
 'brar',
 'cagr',
 'calmar_ratio',
 'candle_color',
 'candles',
 'category_files',
 'cci',
 'cdl',
 'cdl_doji',
 'cdl_inside',
 'cdl_pattern',
 'cdl_z',
 'cfo',
 'cg',
 'chop',
 'cksp',
 'cmf',
 'cmo',
 'combination',
 'coppock',
 'core',
 'cpu_count',
 'cross',
 'cr

In [13]:
dir(ta.yf)

['__annotations__',
 '__builtins__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [14]:
nifty_bees= ta.yf("NIFTYBEES.NS")


In [15]:
nifty_bees.tail()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-08-10 00:00:00+05:30,217.270004,217.270004,215.009995,215.330002,2100995,0.0,0.0
2023-08-11 00:00:00+05:30,217.210007,217.210007,214.250000,214.399994,2385921,0.0,0.0
2023-08-14 00:00:00+05:30,217.820007,217.820007,211.649994,214.509995,2878972,0.0,0.0
2023-08-16 00:00:00+05:30,218.470001,218.470001,212.679993,214.570007,1604526,0.0,0.0
2023-08-17 00:00:00+05:30,218.169998,218.169998,208.149994,214.210007,383666,0.0,0.0


In [16]:
help(nifty_bees.ta.strategy)


Help on method strategy in module pandas_ta.core:

strategy(*args, **kwargs) method of pandas_ta.core.AnalysisIndicators instance
    Strategy Method
    
    An experimental method that by default runs all applicable indicators.
    Future implementations will allow more specific indicator generation
    with possibly as json, yaml config file or an sqlite3 table.
    
    
    Kwargs:
        chunksize (bool): Adjust the chunksize for the Multiprocessing Pool.
            Default: Number of cores of the OS
        exclude (list): List of indicator names to exclude. Some are
            excluded by default for various reasons; they require additional
            sources, performance (td_seq), not a ohlcv chart (vp) etc.
        name (str): Select all indicators or indicators by
            Category such as: "candles", "cycles", "momentum", "overlap",
            "performance", "statistics", "trend", "volatility", "volume", or
            "all". Default: "all"
        ordered (bool): W

In [17]:
help(nifty_bees.ta)


Help on AnalysisIndicators in module pandas_ta.core object:

class AnalysisIndicators(BasePandasObject)
 |  AnalysisIndicators(pandas_obj)
 |  
 |  This Pandas Extension is named 'ta' for Technical Analysis. In other words,
 |  it is a Numerical Time Series Feature Generator where the Time Series data
 |  is biased towards Financial Market data; typical data includes columns
 |  named :"open", "high", "low", "close", "volume".
 |  
 |  This TA Library hopefully allows you to apply familiar and unique Technical
 |  Analysis Indicators easily with the DataFrame Extension named 'ta'. Even
 |  though 'ta' is a Pandas DataFrame Extension, you can still call Technical
 |  Analysis indicators individually if you are more comfortable with that
 |  approach or it allows you to easily and automatically apply the indicators
 |  with the strategy method. See: help(ta.strategy).
 |  
 |  By default, the 'ta' extension uses lower case column names: open, high,
 |  low, close, and volume. You can ove